# Ground state of the Heisenberg model

## Overview

This notebook will give an example how to use Renormalizer to calculate the ground state energy of the open boundary spin 1/2 1-D Heisenberg model.

$$
H = J \sum_i [S_i^z S_{i+1}^z + \frac{1}{2}(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)]
$$

Exact result via Bethe Anstatz:

|$L$ |      $E/J$|
|----|-----------|
|16  |-6.9117371455749|
|24  |-10.4537857604096|
|32  |-13.9973156182243|
|48  |-21.0859563143863|
|64  |-28.1754248597421|


## Setup

In [1]:
from renormalizer import Model, Op, BasisHalfSpin,  Mps, Mpo, optimize_mps

2025-11-07 07:36:34,970[INFO] Use NumPy as backend


2025-11-07 07:36:34,971[INFO] numpy random seed is 9012


2025-11-07 07:36:34,971[INFO] random seed is 1092


2025-11-07 07:36:34,981[INFO] Git Commit Hash: 190c65bf4368526ca515f9397f0e9be7762e06c6


2025-11-07 07:36:34,982[INFO] use 64 bits


## Define the Model
In Renormalizer, models are defined by the Hamiltonian terms and the a list of basis sets. The basis also defines the ordering in DMRG.

The spin operators can be represented by Pauli operators
$$
S^+ = \sigma^+ 
$$
$$
S^- = \sigma^- 
$$
$$
S^{\{x,y,z\}} = \frac{1}{2} \sigma^{\{x,y,z\}}
$$

In [2]:
# define the # of spins
nspin = 32

# define the model

ham_terms = []
for ispin in range(nspin-1):
    op1 = Op("sigma_z sigma_z", [ispin, ispin+1], 1.0/4)
    op2 = Op("sigma_+ sigma_-", [ispin, ispin+1], 1.0/2)
    op3 = Op("sigma_- sigma_+", [ispin, ispin+1], 1.0/2)
    ham_terms.extend([op1, op2, op3])

# set the spin order and local basis
basis = [BasisHalfSpin(i) for i in range(nspin)]
# construct Hamiltonian MPO
model = Model(basis, ham_terms)
mpo = Mpo(model)
print(f"mpo_bond_dims:{mpo.bond_dims}")

2025-11-07 07:36:35,085[DEBUG] # of operator terms: 93


2025-11-07 07:36:35,085[DEBUG] Input operator terms: 93


2025-11-07 07:36:35,087[DEBUG] After combination of the same terms: 93


2025-11-07 07:36:35,088[DEBUG] symbolic mpo algorithm: qr


mpo_bond_dims:[1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1]


## DMRG Sweeps

In [3]:
# set the sweep paramter
M = 30
# sweep for 5 rounds. For each round, the bond dimension is M.
# in the first round, 20% of the kept states are chosen randomly to avoid
# stucking in local minimum.
procedure = [[M, 0.2], [M, 0], [M, 0], [M,0], [M,0]]

# initialize a random MPS
qntot = 0
mps = Mps.random(model, qntot, M)

mps.optimize_config.procedure = procedure
mps.optimize_config.method = "2site"

# optimize MPS
energies, _ = optimize_mps(mps.copy(), mpo)
print("gs energy:", min(energies))

2025-11-07 07:36:35,158[INFO] optimization method: 2site


2025-11-07 07:36:35,159[INFO] e_rtol: 1e-06


2025-11-07 07:36:35,159[INFO] e_atol: 1e-08


2025-11-07 07:36:35,160[INFO] procedure: [[30, 0.2], [30, 0], [30, 0], [30, 0], [30, 0]]


2025-11-07 07:36:35,191[DEBUG] isweep: 0


...

2025-11-07 07:36:41,246[DEBUG] energy: -13.99731500742738


2025-11-07 07:36:41,248[DEBUG] 3 sweeps are finished, lowest energy = -13.9973150902116


2025-11-07 07:36:41,249[INFO] DMRG has converged!


2025-11-07 07:36:41,287[INFO] mps current size: 329.7KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 16, 8, 4, 2, 1]


gs energy: -13.9973150902116
